# ALGORITHM 1


In [1]:
import numpy as np
import timeit
n=10000
sigma = 1000
def generate_data():
    l = np.random.uniform(10, 50, size=n)
    c = np.random.uniform(10, 50, size=n)
    x_bar = np.random.uniform(1, 10, size=n)
    #y_bar = np.random.uniform(1, c-1, size=n)
    y_bar = [np.random.uniform(1, c_i - 1) for c_i in c]
    lbd = np.random.uniform(5, 10, size=n)
    mu = np.random.uniform(1, 150, size=n)
    input_data= {
        "l": l,
        "c": c,
        "x_bar": x_bar,
        "y_bar": y_bar,
        "lbd": lbd,
        "mu": mu,
    }
    return input_data
input_data=generate_data()
Budget = np.sum(input_data["x_bar"] * input_data["lbd"] + input_data["mu"])/100
def calculate_T(input_data):
    l=input_data["l"]
    c=input_data["c"]
    x_bar=input_data["x_bar"]
    y_bar=input_data["y_bar"]
    lbd = input_data["lbd"]
    mu=input_data["mu"]
    T = l + sigma/c
    T_max = l+ x_bar + sigma/(c - y_bar)
    T_x = l+ x_bar + sigma/c
    T_y = l + sigma/(c - y_bar)
    T_lbd_mu = l + mu/lbd + sigma/c
    output_T= {
        "T": T,
        "T_max": T_max,
        "T_x": T_x,
        "T_y": T_y,
        "T_lbd_mu": T_lbd_mu,

    }
    return output_T
output_T=calculate_T(input_data)


In [2]:
def algorithm_alg1(input_data,output_T,Budget):
  alpha=np.min(output_T["T"])
  beta=np.min(output_T["T_max"])
  def determine_indices(input_data,output_T):
      lbd=input_data["lbd"]
      mu=input_data["mu"]
      x_bar=input_data["x_bar"]
      T_x=output_T["T_x"]
      T_y=output_T["T_y"]
      T_lbd_mu=output_T["T_lbd_mu"]
      I_1 = np.where((x_bar * lbd <= mu) & (T_y <= T_x ))[0]
      I_2 = np.where((x_bar * lbd <= mu) & (T_y > T_x))[0]
      I_3 = np.where((x_bar * lbd > mu) & (T_y <= T_lbd_mu ))[0]
      I_4 = np.where((x_bar * lbd > mu) & (T_y > T_lbd_mu ))[0]
      output_indices={
          "I_1":I_1,
          "I_2":I_2,
          "I_3":I_3,
          "I_4":I_4,
      }
      return output_indices
  output_indices = determine_indices(input_data,output_T)
  def C_breakpoint(i,output_T,output_indices):
      T_x=output_T["T_x"]
      T_y=output_T["T_y"]
      T_lbd_mu=output_T["T_lbd_mu"]
      I_1=output_indices["I_1"]
      I_2=output_indices["I_2"]
      I_3=output_indices["I_3"]
      I_4=output_indices["I_4"]
      C_break=[]
      if i in I_1:
        C_break.append(T_x[i])
      elif i in I_2:
        C_break.append(T_x[i])
        C_break.append(T_y[i])
      elif i in I_3:
        C_break.append(T_x[i])
      elif i in I_4:
        C_break.append(T_lbd_mu[i])
        C_break.append(T_y[i])
      return C_break
  # All C_breakpoints
  all_C_breakpoints = []
  for i in range(len(input_data["l"])):
      all_C_breakpoints.append(C_breakpoint(i, output_T, output_indices))
  # Fnd breakpoint H
  B = set()
  for sublist in all_C_breakpoints:
      B.update(sublist)
  B.update(output_T["T"])
  B.update([alpha,beta])
  # strict it to [alpha, beta]
  break_points= np.array(list(B))
  H = []
  for t in break_points :
      if alpha<=t<= beta:
          H.append(t)
  H= np.sort(H)
  #Cost function
  def get_cost(i, t, input_data, output_T, output_indices):
      lbd = input_data["lbd"]
      mu = input_data["mu"]
      T=output_T["T"]
      T_x=output_T["T_x"]
      T_y=output_T["T_y"]
      T_lbd_mu=output_T["T_lbd_mu"]
      I_1=output_indices["I_1"]
      I_2=output_indices["I_2"]
      I_3=output_indices["I_3"]
      I_4=output_indices["I_4"]
      # Xử lý tính toán giá trị cost
      if i in I_1:
          if t<= T[i]:
              cost = 0
          elif T[i] < t <= T_x[i]:
              cost = lbd[i] * (t - T[i])
          else:
              cost = mu[i] + lbd[i] * (t - T_y[i])
      elif i in I_2:
          if t<= T[i]:
              cost = 0
          elif T[i] < t <= T_x[i]:
              cost = lbd[i] * (t - T[i])
          elif T_x[i] < t <= T_y[i]:
              cost=mu[i]
          else:
              cost = mu[i] + lbd[i] * (t - T_y[i])
      elif i in I_3:
          if t<= T[i]:
              cost = 0
          elif T[i] < t <= T_x[i]:
              cost = lbd[i] * (t - T[i])
          else:
              cost = mu[i] + lbd[i] * (t - T_y[i])
      elif i in I_4:
          if t<= T[i]:
              cost = 0
          elif T[i] <t <= T_lbd_mu[i]:
              cost = lbd[i] * (t - T[i])
          elif T_lbd_mu[i] < t <= T_y[i]:
              cost=mu[i]
          else:
              cost = mu[i] + lbd[i] * (t - T_y[i])
      return cost
  def total_cost(t, input_data, output_T, output_indices):
      total_cost = 0
      for i in range(len(input_data["l"])):
          cost = get_cost(i, t, input_data, output_T, output_indices)
          total_cost += cost
      return total_cost
# Algorithm_1
  def binary_search(H, Budget):
      a = 0
      b = len(H) - 1

      while a <= b:
          k = (a + b) // 2
          Omega_k = total_cost(t=H[k], input_data=input_data, output_T=output_T, output_indices=output_indices)
          Omega_k_ep = total_cost(t=H[k] + 1e-5, input_data=input_data, output_T=output_T, output_indices=output_indices)

          if Omega_k == Budget:
              return k

          if Omega_k < Budget and Omega_k_ep > Budget:
              return k

          if Omega_k < Budget and Omega_k_ep < Budget:
              a = k + 1
          else:
              b = k - 1

      return k
  optimal_index = binary_search(H, Budget)
# Find optimal solution
  def find_t_optimal(Budget, optimal_index, H):
      Omega_i = total_cost(t=H[optimal_index], input_data=input_data, output_T=output_T, output_indices=output_indices)
      Omega_ep = total_cost(t=H[optimal_index] + 1e-5, input_data=input_data, output_T=output_T, output_indices=output_indices)

      # If Omega(optimal) = Budget, return t_optimal = optimal
      if Omega_i == Budget:
          return H[optimal_index]

      # If Omega(optimal) < Budget and Omega(optimal + epsilon) > Budget
      elif Omega_i < Budget and Omega_ep > Budget:
          return H[optimal_index]

      # If Omega(optimal) < Budget and Omega(optimal + epsilon) < Budget
      elif Omega_i < Budget and Omega_ep < Budget:
          H_optimal = H[optimal_index]
          if optimal_index == len(H) - 1:
            H_next = H[optimal_index]
          else:
            H_next = H[optimal_index + 1]

          def find_theta_within_range(start, end, Budget):
              epsilon_theta = 0.001
              lower_theta = start
              upper_theta = end

              while abs(upper_theta - lower_theta) > epsilon_theta:
                  mid_theta = (lower_theta + upper_theta) / 2
                  current_budget_theta = total_cost(mid_theta, input_data=input_data, output_T=output_T, output_indices=output_indices)

                  if abs(current_budget_theta - Budget) < epsilon_theta:
                      return mid_theta

                  if current_budget_theta < Budget:
                      lower_theta = mid_theta
                  else:
                      upper_theta = mid_theta

              return (lower_theta + upper_theta) / 2

          t_optimal = find_theta_within_range(H_optimal, H_next, Budget)
          return t_optimal

  t_optimal = find_t_optimal(Budget, optimal_index, H)
# Update indices
  def update_indices(t,group, output_T, output_indices):
      """
      group = "I_1", "I_2", "I_3", "I_4"
      """

      T = output_T["T"]
      I_1 = output_indices["I_1"]
      I_2 = output_indices["I_2"]
      I_3 = output_indices["I_3"]
      I_4 = output_indices["I_4"]
      if group == "I_1":
          # i in IU: U <= t
          res = [i for i in I_1 if T[i] < t]
      elif group == "I_2":
          # i in IU: U <= t
          res = [i for i in I_2 if T[i] < t]
      elif group == "I_3":
          # i in IU: U <= t
          res = [i for i in I_3 if T[i] < t]
      elif group == "I_4":
          # i in IU: U <= t
          res = [i for i in I_4 if T[i] < t]
      return res
  I_1_update=update_indices(t=t_optimal, group="I_1",output_T=output_T, output_indices=output_indices)
  I_2_update=update_indices(t=t_optimal, group="I_2",output_T=output_T, output_indices=output_indices)
  I_3_update=update_indices(t=t_optimal, group="I_3",output_T=output_T, output_indices=output_indices)
  I_4_update=update_indices(t=t_optimal, group="I_4",output_T=output_T, output_indices=output_indices)
# Return modification
  def solution_for_group(i, t, output_T, input_data, group):
      T = output_T["T"]
      T_x = output_T["T_x"]
      T_y = output_T["T_y"]
      T_lbd_mu = output_T["T_lbd_mu"]
      y_bar = input_data["y_bar"]

      if group == "I_1_update":
          if t< T[i]:
            x_i = 0
            y_i = 0
          elif T[i] < t <= T_x[i]:
            x_i = t - T[i]
            y_i = 0
          else:
            x_i = t - T_y[i]
            y_i = y_bar[i]
      elif group == "I_2_update":
          if t< T[i]:
            x_i = 0
            y_i = 0
          elif T[i] < t <= T_x[i]:
              x_i = t - T[i]
              y_i = 0
          elif T_x[i] < t <= T_y[i]:
              x_i = 0
              y_i= y_bar[i]
          else:
              x_i = t - T_y[i]
              y_i= y_bar[i]
      elif group == "I_3_update":
          if t< T[i]:
            x_i = 0
            y_i = 0
          elif T[i] <t <= T_x[i]:
              x_i = t - T[i]
              y_i = 0
          else:
              x_i = t - T_y[i]
              y_i = y_bar[i]
      elif group == "I_4_update":
          if t< T[i]:
            x_i = 0
            y_i = 0
          elif T[i] <t <= T_lbd_mu[i]:
              x_i = t - T[i]
              y_i = 0
          elif T_lbd_mu[i] < t <= T_y[i]:
              x_i = 0
              y_i = y_bar[i]
          else:
              x_i = t - T_y[i]
              y_i = y_bar[i]
      return x_i, y_i

  
  x = np.zeros(len(input_data["l"]))
  y = np.zeros(len(input_data["l"]))

  
  for i in I_1_update:
      x_i, y_i = solution_for_group(i, t=t_optimal, output_T=output_T, input_data=input_data, group="I_1_update")
      x[i] = x_i
      y[i] = y_i
  for i in I_2_update:
      x_i, y_i = solution_for_group(i, t=t_optimal, output_T=output_T, input_data=input_data, group="I_2_update")
      x[i] = x_i
      y[i] = y_i
  for i in I_3_update:
      x_i, y_i = solution_for_group(i, t=t_optimal, output_T=output_T, input_data=input_data, group="I_3_update")
      x[i] = x_i
      y[i] = y_i
  for i in I_4_update:
      x_i, y_i = solution_for_group(i, t=t_optimal, output_T=output_T, input_data=input_data, group="I_4_update")
      x[i] = x_i
      y[i] = y_i
  return x,y
result_alg1= algorithm_alg1(input_data,output_T,Budget)

In [4]:
# Number of runs
num_runs = 50

# Calculate the running time of Algorithm 1
times_alg1 = timeit.repeat(lambda: algorithm_alg1(input_data,output_T,Budget), number=1, repeat=num_runs)

# Average running time of Algorithm 1
time_avg_alg1 = sum(times_alg1) / num_runs
print(time_avg_alg1)